In [11]:
import torch
import re
import os
import json
import pickle
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from dotenv import load_dotenv

### Dotenv

In [12]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

### DataFrames

In [13]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

### Trait Dicts

In [14]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

### Text Data

In [15]:
paragraph_folder = "../../../data/OpenAI/DescriptionSnippets/Paragraphs/"
sentence_folder = "../../../data/OpenAI/DescriptionSnippets/Sentences/"

caribbean_jsons_paras = glob.glob(F"{paragraph_folder}c*")
caribbean_jsons_sents = glob.glob(F"{sentence_folder}c*")

caribbean_jsons_paras.sort()
caribbean_jsons_sents.sort()

### Single Species Testing

In [16]:
with open(caribbean_jsons_paras[22], 'r') as f:
  caribbean_species_para_test = json.load(f)

with open(caribbean_jsons_sents[22], 'r') as f:
  caribbean_species_sent_test = json.load(f)

In [17]:
species = list(caribbean_species_sent_test.keys())[0]
print(species)

Hippomane mancinella


In [18]:
text_sent = ' '.join(caribbean_species_sent_test[species])
text_paras = ' '.join(caribbean_species_para_test[species])

text_paras == text_sent

True

### Prompting per Trait

In [19]:
def create_prompt(paragraph, traits, question):

    prompt = F"""
    Text:
    {paragraph}

    Trait with possible options:
    {traits}

    {question}
    """

    return prompt

def combine_words_with_capital(string):
    # remove non-alphanumeric characters
    string = re.sub(r'[^\w\s]', '', string)  
    # combine words with capitalization
    string = ''.join(word.capitalize() for word in string.split())  
    return string


def query_ChatGPT(prompt):
    messages = [
    {"role": "user", "content": prompt}
    ]

    # Generate a response
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

    return completion

In [20]:
caribbean_traits_dict = {k: caribbean_traits_dict[k][0:] for k in list(caribbean_traits_dict)[:3]}
caribbean_traits_dict

{'Life form': ['liana', 'tree'],
 'Leaf position': ['alternate',
  'alternate, opposite',
  'opposite',
  'opposite, whorls of 3',
  'opposite, whorls of 3, alternate'],
 'Leaf composition': ['3 palmate',
  '3-5 palmate',
  '3-5 pinnate',
  '3-5 pinnate, entire',
  '5-11 pinnate',
  '5-9 pinnate',
  'bi-pinnate, 2 leaflets per jug',
  'bi-pinnate, 20-40 leaflets per jug',
  'bi-pinnate, 30-32 leaflets per jug',
  'entire',
  'pinnate, 4-6 leaflets',
  'pinnate, 6-8 leaflets']}

In [70]:
folder_prompts = "../../../data/OpenAI/PromptsResults/"
folder_species = species.replace(' ', '_')


# For spss in species:

try:
    os.makedirs(F"{folder_prompts}{folder_species}")
except FileExistsError:
    # directory already exists
    pass

for trait, trait_options in tqdm(caribbean_traits_dict.items()):

    ### PROMPT ###
    
    # Question
    question = F"""Can you select the correct value(s) for the '{trait}' trait from the text?
    Only return the correct value(s) or select 'Something else:' and fill in your finding(s)."""
    
    # List Options
    # options = "\n".join([F"\t- {s}" for s in trait_options])
    # options += "\n\t- Nothing mentioned\n\t- Something else: "
    options = trait_options + ["Nothing mentiond in the next", "Someting else"]

    # Text
    text = text_paras

    question = F"""
    Question: 
    {question}
    
    Values: 
    {options}
    """
    ### PROMPT ###

    # messages = [
    #     {"role": "assistant", "content": text},
    #     {"role": "user", "content": question}
    # ]

    # # Generate a response
    # completion = openai.ChatCompletion.create(
    #     model = "gpt-3.5-turbo",
    #     messages = messages,
    # )

    # file_name = combine_words_with_capital(trait)
    # with open(F"{folder_prompts}{folder_species}/{file_name}.json", 'w') as fp:
    #     json.dump(completion, fp)

100%|██████████| 3/3 [00:03<00:00,  1.29s/it]
